In [8]:
%load_ext autoreload
%autoreload 2

import psutil
import dask.distributed
import rioxarray
import numpy as np
import xarray as xr
from odc.stac import stac_load
from pystac_client import Client
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="angle from rectified to skew grid parameter lost in conversion to CF")

# Get the available memory in gigabytes (to later adapt chunks size)

available_memory = psutil.virtual_memory().available
available_memory_gb = available_memory / (1024 ** 3)

print(f"Available memory: {available_memory_gb:.2f} GB")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Available memory: 4.23 GB


In [2]:
client = dask.distributed.Client()

catalog = Client.open("https://explorer.digitalearth.africa/stac")


In [5]:
import pandas as pd
# Search the catalog for all available products
collections = catalog.get_collections()

# List the collections (products) available in the catalog
product_names = [collection.id for collection in collections]

descriptions = []
# Get the descriptions and print a full list
for pi in product_names:
    collection = catalog.get_collection(pi)
    descriptions.append(collection.description)
    # print(f"{collection.id} - {collection.description}")

df = pd.DataFrame({'products': product_names, 'description': descriptions})
print(df)

/Users/pohle/miniconda3/envs/pystac_example/lib/python3.11/site-packages/pystac_client/client.py:799: MissingLink: No link with rel='data' could be found on this Client.
  href = self._get_href("data", data_link, "collections")


                   products                                        description
0        alos_palsar_mosaic  ALOS/PALSAR and ALOS-2/PALSAR-2 annual mosaic ...
1             cci_landcover           ESA Climate Change Initiative Land Cover
2            cgls_landcover  Copernicus Global Land Service, Land Use/Land ...
3     cgls_lwq100_2019_2024  Copernicus Global Land Service – Lake Water Qu...
4      cgls_lwq100_2024_nrt  Copernicus Global Land Service – Lake Water Qu...
..                      ...                                                ...
95  wofs_ls_summary_alltime   Water Observations from Space Alltime Statistics
96   wofs_ls_summary_annual    Water Observations from Space Annual Statistics
97                 wsf_2015  The World Settlement Footprint WSF 2015 versio...
98                 wsf_2019  The World Settlement Footprint (WSF) 2019 is a...
99            wsf_evolution  The World Settlement Footprint (WSF) Evolution...

[100 rows x 2 columns]


In [ ]:
for n in df.products:
    print(n)

alos_palsar_mosaic
cci_landcover
cgls_landcover
crop_mask
crop_mask_central
crop_mask_eastern
crop_mask_indian_ocean
crop_mask_northern
crop_mask_sahel
crop_mask_southeast
crop_mask_southern
crop_mask_western
dem_cop_30
dem_cop_90
dem_srtm
dem_srtm_deriv
esa_worldcover
esa_worldcover_2020
esa_worldcover_2021
fc_ls
fc_ls_summary_annual
gm_ls5_ls7_annual
gm_ls5_ls7_annual_lowres
gm_ls8_annual
gm_ls8_annual_lowres
gm_ls8_ls9_annual
gm_ls8_ls9_annual_lowres
gm_s2_annual
gm_s2_annual_lowres
gm_s2_rolling
gm_s2_semiannual
gm_s2_semiannual_lowres
gmw
io_lulc
isda_soil_bedrock_depth
isda_soil_bulk_density
isda_soil_carbon_total
isda_soil_clay_content
isda_soil_sand_content
isda_soil_silt_content
jers_sar_mosaic
landsat_c2l2_ar
ls5_sr
ls5_st
ls7_sr
ls7_st
ls8_sr
ls8_st
ls9_sr
ls9_st
maxar_morocco_earthquake
maxar_morocco_earthquake_4bands
maxar_morocco_earthquake_8bands
nasadem
ndvi_anomaly
ndvi_climatology_ls
pc_s2_annual
rainfall_chirps_daily
rainfall_chirps_monthly
s1_rtc
s2_l2a
s2_l2a_c1
s3

In [16]:
# Some information, e.g. the available dates, can be extracted from the filename information
# Other, more detailed information are provided on the website of the SDC (https://explorer.swissdatacube.org/products)

# Get the collection client
collection = catalog.get_collection("ndvi_anomaly")

# Initialize a list to hold the extracted values
acquisition_dates = []

# Iterate over the links
for link in collection.links:
    if link.rel == "child":  # Filter for links with the 'child' relationship
        # Extract the last part of the href
        last_part = link.href.split('/')[-1]  # Get the last segment
        acquisition_dates.append(last_part)

# Print the extracted acquisition dates
print("Acquisition Dates:", acquisition_dates)

Acquisition Dates: ['2017-1', '2017-2', '2017-3', '2017-4', '2017-5', '2017-6', '2017-7', '2017-8', '2017-9', '2017-10', '2017-11', '2017-12', '2018-1', '2018-2', '2018-3', '2018-4', '2018-5', '2018-6', '2018-7', '2018-8', '2018-9', '2018-10', '2018-11', '2018-12', '2019-1', '2019-2', '2019-3', '2019-4', '2019-5', '2019-6', '2019-7', '2019-8', '2019-9', '2019-10', '2019-11', '2019-12', '2020-1', '2020-2', '2020-3', '2020-4', '2020-5', '2020-6', '2020-7', '2020-8', '2020-9', '2020-10', '2020-11', '2020-12', '2021-1', '2021-2', '2021-3', '2021-4', '2021-5', '2021-6', '2021-7', '2021-8', '2021-9', '2021-10', '2021-11', '2021-12', '2022-1', '2022-2', '2022-3', '2022-4', '2022-5', '2022-6', '2022-7', '2022-8', '2022-9', '2022-10', '2022-11', '2022-12', '2023-1', '2023-2', '2023-3', '2023-4', '2023-5', '2023-6', '2023-7', '2023-8', '2023-9', '2023-10', '2023-11', '2023-12', '2024-1', '2024-2', '2024-3', '2024-4', '2024-5', '2024-6', '2024-7', '2024-8', '2024-9', '2024-10', '2024-11', '2025-1

In [17]:
client.close()